<a href="https://colab.research.google.com/github/yunyicheng/DAD-Oligonucleotides/blob/main/Data_optimized_Assembly_Design_for_Oligonucleotides.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction

Deep mutational scanning uses thousands of oligonucleotides in one reaction tube, and the success of the reaction is dependent on on-target and off-target reactions. These on-target and off-target reactions have previously been characterized in details. This tool maximizes on-target reactions and minimizes off-target reactions through the use of gradient descent and other necessary rules for deep-mutational scanning. 

# 2. Data Processing

## 2.1 Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio.Seq import Seq

## 2.2 Process Target gene

In [ ]:
# split the gene by codons to make sure cassettes inserted at codon boundaries
# target_gene is S.cerevisiae Rad27, could be altered for other genes of interests
target_gene = 'ATGGGTATTAAAGGTTTGAATGCAATTATATCGGAACATGTTCCCTCTGCTATCAGGAAAAGCGATATCAAGAGCTTTTTTGGCAGAAAGGTTGCCATCGATGCCTCTATGTCTCTATATCAGTTTTTAATTGCTGTAAGACAGCAAGACGGTGGGCAGTTGACCAATGAAGCCGGTGAAACAACGTCACACTTGATGGGTATGTTTTATAGGACACTGAGAATGATTGATAACGGTATCAAGCCTTGTTATGTCTTCGACGGCAAACCTCCAGATTTGAAATCTCATGAGTTGACAAAGCGGTCTTCAAGAAGGGTGGAAACAGAAAAAAAACTGGCAGAGGCAACAACAGAATTGGAAAAGATGAAGCAAGAAAGAAGATTGGTGAAGGTCTCAAAAGAGCATAATGAAGAAGCCCAAAAATTACTAGGACTAATGGGAATCCCATATATAATAGCGCCAACGGAAGCTGAGGCTCAATGTGCTGAGTTGGCAAAGAAGGGAAAGGTGTATGCCGCAGCAAGTGAAGATATGGACACACTCTGTTATAGAACACCCTTCTTGTTGAGACATTTGACTTTTTCAGAGGCCAAGAAGGAACCGATTCACGAAATAGATACTGAATTAGTTTTGAGAGGACTCGACTTGACAATAGAGCAGTTTGTTGATCTTTGCATAATGCTTGGTTGTGACTACTGTGAAAGCATCAGAGGTGTTGGTCCAGTGACAGCCTTAAAATTGATAAAAACGCATGGATCCATCGAAAAAATCGTGGAGTTTATTGAATCTGGGGAGTCAAACAACACTAAATGGAAAATCCCAGAAGACTGGCCTTACAAACAAGCAAGAATGCTGTTTCTTGACCCTGAAGTTATAGATGGTAACGAAATAAACTTGAAATGGTCGCCACCAAAGGAGAAGGAACTTATCGAGTATTTATGTGATGATAAGAAATTCAGTGAAGAAAGAGTTAAATCTGGTATATCAAGATTGAAAAAAGGCTTGAAATCTGGCATTCAGGGTAGGTTAGATGGGTTCTTCCAAGTGGTGCCTAAGACAAAGGAACAGCTGGCTGCTGCGGCGAAAAGAGCACAAGAAAATAAAAAATTGAACAAAAATAAGAATAAAGTCACAAAGGGAAGAAGATGA'
target_gene = [target_gene[i:i+3] for i in range(0, len(target_gene), 3)]
print(target_gene)
print(len(target_gene))

['ATG', 'GGT', 'ATT', 'AAA', 'GGT', 'TTG', 'AAT', 'GCA', 'ATT', 'ATA', 'TCG', 'GAA', 'CAT', 'GTT', 'CCC', 'TCT', 'GCT', 'ATC', 'AGG', 'AAA', 'AGC', 'GAT', 'ATC', 'AAG', 'AGC', 'TTT', 'TTT', 'GGC', 'AGA', 'AAG', 'GTT', 'GCC', 'ATC', 'GAT', 'GCC', 'TCT', 'ATG', 'TCT', 'CTA', 'TAT', 'CAG', 'TTT', 'TTA', 'ATT', 'GCT', 'GTA', 'AGA', 'CAG', 'CAA', 'GAC', 'GGT', 'GGG', 'CAG', 'TTG', 'ACC', 'AAT', 'GAA', 'GCC', 'GGT', 'GAA', 'ACA', 'ACG', 'TCA', 'CAC', 'TTG', 'ATG', 'GGT', 'ATG', 'TTT', 'TAT', 'AGG', 'ACA', 'CTG', 'AGA', 'ATG', 'ATT', 'GAT', 'AAC', 'GGT', 'ATC', 'AAG', 'CCT', 'TGT', 'TAT', 'GTC', 'TTC', 'GAC', 'GGC', 'AAA', 'CCT', 'CCA', 'GAT', 'TTG', 'AAA', 'TCT', 'CAT', 'GAG', 'TTG', 'ACA', 'AAG', 'CGG', 'TCT', 'TCA', 'AGA', 'AGG', 'GTG', 'GAA', 'ACA', 'GAA', 'AAA', 'AAA', 'CTG', 'GCA', 'GAG', 'GCA', 'ACA', 'ACA', 'GAA', 'TTG', 'GAA', 'AAG', 'ATG', 'AAG', 'CAA', 'GAA', 'AGA', 'AGA', 'TTG', 'GTG', 'AAG', 'GTC', 'TCA', 'AAA', 'GAG', 'CAT', 'AAT', 'GAA', 'GAA', 'GCC', 'CAA', 'AAA', 'TTA', 'CTA'

## 2.3 Load Overhang Fidelity Chart

In [ ]:
# check if path to chart is valid
path_name = '/content/overhang_fidelity.xlsx'
while not os.path.isfile(path_name):
    path_name = input("Whoops! No such file! Please enter the name of the file you'd like to use.")
# read the excel file into pandas dataframe
df = pd.read_excel(path_name, sheet_name='S1 Table. BsaI-HFv2', engine='openpyxl')
# set overhang as indices for indexing the chart by row + col names
df.set_index("Overhang", inplace = True)
print (df)
print(df.loc['AAAA']['TTTT'])

          AAAA  AAAC  AAAG  AAAT  AACA  AACC  AACG  AACT  AAGA  AAGC  ...  \
Overhang                                                              ...   
TTTT       635     8    40    16     2     0     0     0     7     0  ...   
GTTT         3   476     4    45     0    20     0     1     0    15  ...   
CTTT         1     1   596     2     0     0    18     0     0     0  ...   
ATTT         8     4     1   642     0     0     0     6     0     1  ...   
TGTT         0     0     0     0   493    16    64    56     2     0  ...   
...        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
ACAA         0     0     0     0     0     0     0     0     0     0  ...   
TAAA         0     0     0     0     0     0     0     0     0     0  ...   
GAAA         0     0     0     0     0     0     0     0     0     0  ...   
CAAA         0     0     0     0     0     0     0     0     0     0  ...   
AAAA         0     0     0     0     0     0     0     0     0     0  ...   

# 3. Score Calculation

In [ ]:
overhang_list = []

In [ ]:
# Obtain the score for one cassette location
# TODO: No overhangs used twice
def obtain_score(start, end) -> int:
    # obtain overhang sequences
    head = target_gene[start-2][-1] + target_gene[start-1]
    tail = target_gene[end+1] + target_gene[end+2][0]
    # convert to biopython seq objects
    head = Seq(head)
    tail = Seq(tail)
    # check for reactivity with existing overhangs
    cross_reaction = 0
    for overhang in overhang_list:
      cross_reaction += df.loc[head][overhang] + df.loc[tail][overhang]
    # record overhangs in notebook if it is not in overhang_list
    repitition = 0
    if head not in overhang_list and tail not in overhang_list:
        overhang_list.append(head)
        overhang_list.append(tail)
    else:
        repitition = 1
    # get reverse compliment of overhang
    head_comp = head.complement()
    tail_comp = tail.complement()
    # check for overhang reactivity summation
    head_reactivity = df.loc[head][head_comp]
    tail_reactivity = df.loc[tail][tail_comp]
    pos_success = head_reactivity + tail_reactivity
    # check how much is the oligo length varied from standard length
    delta_len = end - start + 1 - 40
    # check for palindrome
    palindromicity = 0
    if head == head[::-1]:
        palindromicity += 1
    if tail == tail[::-1]:
        palindromicity += 1
    # final score calculation
    final_score = pos_success - palindromicity * 100 - delta_len ** 2 - repitition * 100
    return final_score

In [ ]:
# Initialization

delta_score = float('inf')
stop_criterion = 5
#while delta_score > stop_criterion:
    